In [1]:
from bokeh import plotting as plt
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import pathlib
mcnp_dir = pathlib.Path.cwd() / "mcnp"

# Load secrets

In [3]:
from plumbum import cmd

lookup_secret = lambda field: cmd.secret_tool("lookup", "app", "phymat", "field", field).strip()
user = lookup_secret("user")
domain = lookup_secret("domain")
port = lookup_secret("port")
password = lookup_secret("password")

# Start SSHFS

In [4]:
import pexpect
p = pexpect.spawn('bash')
p.sendline(f'sshfs {user}@{domain}:mcnp {mcnp_dir} -p {port}')
p.expect("password:")
p.sendline(password)
p.sendline('exit')
p.expect_exact(pexpect.EOF)

0

# Define SSH session

In [365]:
from plumbum.machines.paramiko_machine import ParamikoMachine
    
remote = ParamikoMachine(domain, user, port, password, keep_alive=30)
remote._cwd = remote.cwd.chdir('mcnp')
        
mcnp = remote['./mcnp']

def clean():
    z =  [("-I", n) for n in ("mcnp", "examples", "utils", "pt1.tex", "*.ip")] 
    filenames = remote['ls'](*chain(*z)).splitlines()
    if filenames:
        remote['rm'](*filenames)

# Output highlighting

In [366]:
from colorama import Fore
from itertools import chain
import re

replacers = {r'warning\.': Fore.YELLOW, r'fatal error\.': Fore.RED, 
             r'\d+ particles got lost.': Fore.RED}

def _highlight(string, replacers):
    pattern = '|'.join(f'({p})' for p in replacers)
    def replacer(m):
        code = next(v for i, v in enumerate(replacers.values()) if m.groups()[i] is not None)
        return f"{code}{m.group(0)}{Fore.RESET}"
    return re.sub(pattern, replacer, string)
    
def prettify(text):
    print(_highlight(text, replacers))

# Part 1

In [23]:
%%writefile mcnp/1.ip
MESSAGE:

Practical Monte Carlo part 1.
C
C Cells
1 1 7.92 1 -2 3 -4 5 -6 (-7:8:-9:10:-11) $ Walls
2 0 (-1:2:-3:4:-5:6) $ Void
3 2 1.0 7 -8 9 -10 11 -12 $ Water
4 0 7 -8 9 -10 12 -6 $ Air gap (void)

C Surfaces
C Define outer walls
1 PX -5.20
2 PX 5.20
3 PY -10.20
4 PY 10.20
5 PZ 0.0
6 PZ 20.0
C Define XY wall surfaces
7 PX -5.00
8 PX 5.00
9 PY -10.0
10 PY 10.0
C Define Z wall / water surfaces
11 PZ 0.20
12 PZ 19.0

C Cell importance MAP
IMP:N 1 0 1 1         $  s
M1   26000.42c 0.74 24000.42c 0.18 28000.42c 0.08 $ Stainless steel
M2   1001.42c 0.6667  8016.42c 0.3333   $ Pure water
C Tallying energy fluence (nX where X is type (2) and n an ID: {1, 2, ...})
F12:N (1 2)
F22:N (3 4)
C Tally energy bins for all tallies (log 10 space)
E0 1E-9 1E-8 1E-7 1E-6 1E-5 1E-4 1E-3 1E-2 1E-1 1 10
C MC type
MODE N
C Thermal neutron induced fission of 235U at (0,0,20mm)
SDEF POS=0.0 0.0 2.0 ERG=D1
SP1  -3 0.988 2.249
C NUMBER OF PARTICLE HISTORIES TO RUN
NPS  20000
PRDMP 0 0 1 1 0

Overwriting mcnp/1.ip


In [30]:
clean()

prettify(mcnp("inp=1.ip", "mctal=tal1"))

 mcnp     ver=4c2  ld=01/20/01     04/04/18 14:06:59                  
 warning.    1 cell densities are greater than 40 gram/cc.
 imcn   is done
 warning.  neutron energy cutoff is below some cross-section tables.
 warning.     5 cross sections modified by free gas thermal treatment.
 dump    1 on file runtpe     nps =         0    coll =              0
                              ctm =      0.00     nrn =              0
 xact   is done
 dynamic storage =    177270 words,     709080 bytes.      cp0 =  0.00
 run terminated when     20000 particle histories were done.
 warning.  tally  12 tfc bin did not pass  7 of 10 statistical checks.
 warning.     2 of   2 tallies did not pass all 10 statistical checks.
 warning.     1 of   2 tallies were all zeros.
 warning.     1 of   2 tallies had bins with large relative errors.
 dump    2 on file runtpe     nps =     20000    coll =        4631219
                              ctm =      0.10     nrn =       78977929
 tally data written to fi

In [22]:
from mctal import tokenizer
from derp import parse
from mctal import g

tokens = [*tokenizer.tokenize_file("mcnp/tal1")]
mctal = next(iter(parse(g.mctal, tokens)))
for tally in mctal.tallies:
    print(tally.data)

# Part 3

In [380]:
part_3_template =\
"""MESSAGE:
    
C Cells
1 4 -19.2 -3 -2 11 $ Uranium source 1
2 4 -19.2 -4 -2 11 $ Uranium source 2
3 4 -19.2 -5 -2 11 $ Uranium source 3
4 4 -19.2 -6 -2 11 $ Uranium source 4
5 3 {moderator_density} 13 -14 15 -16 11 -12 #1 #2 #3 #4 $ Moderator
6 1 7.92 7 -8 9 -10 17 -18 (-13:14:-15:16:-11) $ Wall
7 0          #1 #2 #3 #4 #5 #6 $ VOID

C Source surfaces
2 PZ 25.2
3 C/Z -20 0 7.5
4 C/Z 0 20 7.5
5 C/Z 20 0 7.5
6 C/Z 0 -20 7.5
C Internal surfaces
7 PX -50.2
8 PX 50.2
9 PY -50.2
10 PY 50.2
11 PZ 0.2
12 PZ 58.0
C Internal surfaces
13 PX -50
14 PX 50
15 PY -50
16 PY 50
17 PZ 0
18 PZ 60

MODE N
KCODE 1000 1.0 200 1000
KSRC 20 0 12.7 -20 0 12.7 0 20 12.7 0 -20 12.7
M1  26000.42c -0.74 24000.42c -0.18 28000.42c -0.08 $ Stainless steel
M2  8016.42c -0.53 14000.42c -0.34 20000.42c -0.10 1001.42c -0.03 $ Concrete
M3  {moderator_def}
M4  92238.42c -{x_238} 92235.42c -{x_235} $ 80% 238U, 20% 235U
IMP:N 1 1 1 1 1 1 0"""

# Define enrichment and water moderator

In [381]:
x_235 = 0.2
source_info = dict(x_235=x_235, x_238=1-x_235)
water_info = dict(moderator_def="1001.42c 2 8016.42c 1",
                  moderator_density=1.0)

In [382]:
clean()

# 3.a

In [383]:
(mcnp_dir/"3.a.ip").write_text(
    part_3_template.format(**water_info, **source_info, description="Water moderated example")
)
prettify(mcnp("inp=3.a.ip", "mctal=tal3.a"))

 mcnp     ver=4c2  ld=01/20/01     04/04/18 22:06:32                  
 m2  8016.42c -0.53 14000.42c -0.34 20000.42c -0.10 1001.42c -0.03 $ C
 warning.  material   2 is not used in the problem.
  total fission nubar data are being used.
 
 warning.    1 materials had unnormalized fractions. print table 40.
 warning.    1 cell densities are greater than 40 gram/cc.
 imcn   is done
 warning.   92235.42c lacks delayed neutron cross sections.
 warning.   92238.42c lacks delayed neutron cross sections.
 warning.  neutron energy cutoff is below some cross-section tables.
 warning.     7 cross sections modified by free gas thermal treatment.
 dump    1 on file runtpe     nps =         0    coll =              0
                              ctm =      0.00     nrn =              0
 source distribution to file srctp           cycle =     0
 xact   is done
 dynamic storage =   1102636 words,    4410544 bytes.      cp0 =  0.01

 cycle    k(col)       ctm     active     k(col)    std dev        f

# 3.c Replace water with graphite

In [ ]:
(mcnp_dir/"3.c.ip").write_text(
    part_3_template.format(**graphite_info, **source_info, description="Graphite moderated example")
)
prettify(mcnp("inp=3.c.ip", "mctal=tal3.c"))

# 3.d Graphite and Uranium increased enrichment

In [ ]:
x_235 = 0.25
source_info = dict(x_235=x_235, x_238=1-x_235)

(mcnp_dir/"3.d.1.ip").write_text(
    part_3_template.format(**water_info, **source_info, description="Water moderated example")
)
prettify(mcnp("inp=3.d.1.ip", "mctal=tal3.d.1"))

(mcnp_dir/"3.d.2.ip").write_text(
    part_3_template.format(**graphite_info, **source_info, description="Graphite moderated example")
)
prettify(mcnp("inp=3.d.2.ip", "mctal=tal3.d.2"))

Lessons learned from MCNP4
* Plotting window doesn't show boundary errors necessarily intuitively 
* KCODE may require specifying upper bound on alloc space (10x n hist here)
* VOL card ignores cell numbers, and instead uses definition order

In [23]:
tokens = [*tokenizer.tokenize_file("mcnp/pt3tal")]

mctal = next(iter(parse(g.mctal, tokens)))
mctal

In [ ]:
!git commit -am "bugfix mctal.py in header" && git push